In [1]:
import os
import sys
import ast
import json
from collections import defaultdict

import cv2
import numpy as np
import pandas as pd
import neuroglancer
from tqdm import tqdm
from skimage import io
import imagesize

HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)
from Controllers.SqlController import SqlController
from lib.FileLocationManager import DATA_PATH, FileLocationManager

In [2]:
animal = 'MD585'
downsample_factor = 32
# OUTPUT_DIR_PATH = os.path.join(os.path.expanduser('~'))
OUTPUT_DIR_PATH = os.path.join('./')
CSV_DIR_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/thumbnail'
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(INPUT))
downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000
structures = sqlController.get_structures_dict()

In [3]:
structures

{'SC': ['Superior colliculus', 18]}

## Get the annotation points

In [4]:
def get_dense_coordinates(coor_list):
    dense_coor_list = []
    # Shortest distance, x, y

    # for x, y in coor_list:
    for i in range(len(coor_list) - 1):
        x, y = coor_list[i]
        x_next, y_next = coor_list[i + 1]

        x_mid = (x + x_next) / 2
        y_mid = (y + y_next) / 2

        dense_coor_list.append([x, y])
        dense_coor_list.append([x_mid, y_mid])

        if i == len(coor_list) - 2:
            dense_coor_list.append([x_next, y_next])
            x, y = coor_list[0]
            x_mid = (x + x_next) / 2
            y_mid = (y + y_next) / 2
            dense_coor_list.append([x_mid, y_mid])

    return dense_coor_list

def get_contours_from_annotations(stack, target_structure, hand_annotations, densify=0):
    MD585_ng_section_min = 83
    num_annotations = len(hand_annotations)
    str_contours_annotation = {}

    for i in range(num_annotations):
        structure = hand_annotations['name'][i]
        side = hand_annotations['side'][i]
        section = hand_annotations['section'][i]
        first_sec = 0
        last_sec = 0

        #if side == 'R' or side == 'L':
        #    structure = structure + '_' + side

        if structure == target_structure:
            vertices = hand_annotations['vertices'][i]

            for i in range(densify):
                vertices = get_dense_coordinates(vertices)

            # Skip sections before the 22nd prep2 section for MD585 as there are clear errors
            if stack == 'MD585' and section < MD585_ng_section_min + 22:
                # vertices = vertices - np.array(MD585_abberation_correction)
                continue
            str_contours_annotation[section] = {}
            str_contours_annotation[section][structure] = {}
            str_contours_annotation[section][structure][1] = vertices

    try:
        first_sec = np.min(list(str_contours_annotation.keys()))
        last_sec = np.max(list(str_contours_annotation.keys()))
    except:
        pass
    return str_contours_annotation, first_sec, last_sec

csvfile = os.path.join(CSV_DIR_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))
hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))
section_structure_vertices = defaultdict(dict)

In [5]:
for structure, values in tqdm(structures.items()):
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=0)
    for section in contour_annotations:
        section_structure_vertices[section][structure] = contour_annotations[section][structure][1]

100%|██████████| 1/1 [00:00<00:00, 26.44it/s]


## Reproduce create_clean transform

In [6]:
section_offset = {}
for file in tqdm(sorted(os.listdir(INPUT))):
    filepath = os.path.join(INPUT, file)
    width, height = imagesize.get(filepath)
    downsampled_shape = np.array((width, height))    
    section = int(file.split('.')[0])
    section_offset[section] = (downsampled_aligned_shape - downsampled_shape) / 2

100%|██████████| 446/446 [00:00<00:00, 15999.89it/s]


In [7]:
section_offset[240]

array([51., 61.])

## Reproduce create_alignment transform

In [8]:
def parse_elastix_parameter_file(filepath, tf_type=None):
    """
    Parse elastix parameter result file.
    """
    def parameter_elastix_parameter_file_to_dict(filename):
        d = {}
        with open(filename, 'r') as f:
            for line in f.readlines():
                if line.startswith('('):
                    tokens = line[1:-2].split(' ')
                    key = tokens[0]
                    if len(tokens) > 2:
                        value = []
                        for v in tokens[1:]:
                            try:
                                value.append(float(v))
                            except ValueError:
                                value.append(v)
                    else:
                        v = tokens[1]
                        try:
                            value = (float(v))
                        except ValueError:
                            value = v
                    d[key] = value
            return d

    d = parameter_elastix_parameter_file_to_dict(filepath)

    if tf_type is None:
        # For alignment composition script
        rot_rad, x_mm, y_mm = d['TransformParameters']
        center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
        # center[1] = d['Size'][1] - center[1]

        xshift = x_mm / d['Spacing'][0]
        yshift = y_mm / d['Spacing'][1]

        R = np.array([[np.cos(rot_rad), -np.sin(rot_rad)],
                      [np.sin(rot_rad), np.cos(rot_rad)]])
        shift = center + (xshift, yshift) - np.dot(R, center)
        T = np.vstack([np.column_stack([R, shift]), [0, 0, 1]])
        return T

    elif tf_type == 'rigid3d':
        p = np.array(d['TransformParameters'])
        center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
        shift = p[3:] / np.array(d['Spacing'])

        thetax, thetay, thetaz = p[:3]
        # Important to use the negative angle.
        cx = np.cos(-thetax)
        cy = np.cos(-thetay)
        cz = np.cos(-thetaz)
        sx = np.sin(-thetax)
        sy = np.sin(-thetay)
        sz = np.sin(-thetaz)
        Rx = np.array([[1, 0, 0], [0, cx, sx], [0, -sx, cx]])
        Ry = np.array([[cy, 0, sy], [0, 1, 0], [-sy, 0, cy]])
        Rz = np.array([[cz, sz, 0], [-sz, cz, 0], [0, 0, 1]])

        R = np.dot(np.dot(Rz, Ry), Rx)
        # R = np.dot(np.dot(Rx, Ry), Rz)
        # The order could be Rx,Ry,Rz - not sure.

        return R, shift, center

    elif tf_type == 'affine3d':
        p = np.array(d['TransformParameters'])
        L = p[:9].reshape((3, 3))
        shift = p[9:] / np.array(d['Spacing'])
        center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
        # shift = center + shift - np.dot(L, center)
        # T = np.column_stack([L, shift])
        return L, shift, center

    elif tf_type == 'bspline3d':
        n_params = d['NumberOfParameters']
        p = np.array(d['TransformParameters'])
        grid_size = d['GridSize']
        grid_spacing = d['GridSpacing']
        grid_origin = d['GridOrigin']

        return L, shift, center

def load_consecutive_section_transform(stack, moving_fn, fixed_fn):
    """
    Load pairwise transform.

    Returns:
        (3,3)-array.
    """
    assert stack is not None
    fileLocationManager = FileLocationManager(stack)
    elastix_output_dir = fileLocationManager.elastix_dir
    param_fp = os.path.join(elastix_output_dir, moving_fn + '_to_' + fixed_fn, 'TransformParameters.0.txt')
    #sys.stderr.write('Load elastix-computed transform: %s\n' % param_fp)
    if not os.path.exists(param_fp):
        raise Exception('Transform file does not exist: %s to %s, %s' % (moving_fn, fixed_fn, param_fp))
    transformation_to_previous_sec = parse_elastix_parameter_file(param_fp)

    return transformation_to_previous_sec

def parse_elastix(animal):
    """
    After the elastix job is done, this goes into each subdirectory and parses the Transformation.0.txt file
    Args:
        animal: the animal
    Returns: a dictionary of key=filename, value = coordinates
    """
    fileLocationManager = FileLocationManager(animal)
    DIR = fileLocationManager.prep
    INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')

    image_name_list = sorted(os.listdir(INPUT))
    anchor_idx = len(image_name_list) // 2
    # anchor_idx = len(image_name_list) - 1
    transformation_to_previous_sec = {}

    for i in range(1, len(image_name_list)):
        fixed_fn = os.path.splitext(image_name_list[i - 1])[0]
        moving_fn = os.path.splitext(image_name_list[i])[0]
        transformation_to_previous_sec[i] = load_consecutive_section_transform(animal, moving_fn, fixed_fn)

    transformation_to_anchor_sec = {}
    # Converts every transformation
    for moving_idx in range(len(image_name_list)):
        if moving_idx == anchor_idx:
            transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)
        elif moving_idx < anchor_idx:
            T_composed = np.eye(3)
            for i in range(anchor_idx, moving_idx, -1):
                T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed
        else:
            T_composed = np.eye(3)
            for i in range(anchor_idx + 1, moving_idx + 1):
                T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed

    return transformation_to_anchor_sec

def create_warp_transforms(transforms, downsample_factor=32):
    def convert_2d_transform_forms(arr):
        return np.vstack([arr, [0, 0, 1]])
    
    transforms_scale_factor = 32 / downsample_factor 
    tf_mat_mult_factor = np.array([[1, 1, transforms_scale_factor], [1, 1, transforms_scale_factor]])
    transforms_to_anchor = {}
    for img_name, tf in transforms.items():
        transforms_to_anchor[img_name] = convert_2d_transform_forms(np.reshape(tf, (3, 3))[:2] * tf_mat_mult_factor) 

    return transforms_to_anchor

transforms = parse_elastix(animal)
warp_transforms = create_warp_transforms(transforms, downsample_factor)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for section, transform in ordered_transforms:
    section_num = int(section.split('.')[0])
    transform = np.linalg.inv(transform)
    section_transform[section_num] = transform

In [12]:
transforms['240.tif']

array([[ 9.99027576e-01,  4.40897031e-02, -1.04921179e+02],
       [-4.40897031e-02,  9.99027576e-01,  3.22309683e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [13]:
section_transform[240]

array([[ 9.99027576e-01, -4.40897031e-02,  1.06240205e+02],
       [ 4.40897031e-02,  9.99027576e-01, -2.75736825e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

## Alignment of annotation coordinates

In [21]:
def transform_create_alignment(points, transform):
    '''
    (x, y) = points[i]
    T = transform
    
    (x', y') = (x * sx + y * ry + tx, x * rx + y * sy + ty)
    'sx': T[0, 0], 'sy': T[1, 1], 'rx': T[1, 0], 'ry': T[0, 1], 'tx': T[0, 2], 'ty': T[1, 2]
    '''
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

aligned_section_structure_polygons = defaultdict(dict)
for section in section_structure_vertices:
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure]) / downsample_factor
        #points = np.array(section_structure_vertices[section][structure]) 
        points = points + section_offset[section] # create_clean offset
        points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
        aligned_section_structure_polygons[section][structure] = [points]

In [22]:
aligned_structures = defaultdict(dict)
for section in section_structure_vertices:
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure])  / downsample_factor
        points = points + section_offset[section] # create_clean offset
        points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
        aligned_structures[structure][section] = points

In [24]:
points = aligned_structures['SC'][136]
print('aligned structures SC At 136', points.shape,np.mean(points, axis=0), np.min(points, axis=0), np.max(points, axis=0))


aligned structures SC At 136 (29, 2) [707.20741699 260.81367919] [698.12488614 251.73713315] [717.20978746 270.69814379]


In [ ]:
def Sort(sub_li):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[1]))    

In [ ]:
for structure, struct_dict in aligned_structures.items():
    for section, points in struct_dict.items():
        if section < 137:
            print(structure, section, np.min(points, axis=0),np.max(points, axis=0) )


In [ ]:
import pickle

vertices_path = os.path.join(OUTPUT_DIR_PATH, f'{animal}_aligned_section_structure_polygons_down{downsample_factor}.pickle')
with open(vertices_path, 'wb') as file:
    pickle.dump(aligned_section_structure_polygons, file)

To this point, aligned_section_structure_polygons variable contains the aligned polygon vertices for each structure in each section. 
From now on, we introduce how to draw these points to numpy array or neuroglancer

## Draw in a numpy volume

In [25]:
def draw_numpy(section_structure_polygons, section_start, section_end):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    #color = get_structure_number(structure)
                    color = 10
#                     cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    for point in polygon:
                        cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

volume = draw_numpy(aligned_section_structure_polygons, 0, num_section)

100%|██████████| 446/446 [00:03<00:00, 115.75it/s]


In [26]:
volume.shape

(1194, 875, 446)

In [ ]:
numpy_path = os.path.join(OUTPUT_DIR_PATH, f'{animal}_annotations_down{downsample_factor}.npy')
with open(numpy_path, 'wb') as file:
    np.save(file, volume)

In [27]:
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=scales), 
        voxel_offset=[0,0,0]
    ),
)

with viewer.txn() as s:
    s.layers['all'] = all_volume_layer

http://127.0.0.1:38637/v/5cfc4eac0b1f3bbcfa0020a247a8804d14871c7f/


In [ ]:
with open('MD589_CVAT_down32.pkl', 'rb') as file:
    cvat_section_structure_polygons = pickle.load(file)
volume = draw_numpy(cvat_section_structure_polygons, 0, num_section)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=scales), 
        voxel_offset=[0,0,0]
    ),
)

with viewer.txn() as s:
    s.layers['cvat'] = all_volume_layer